In [1]:
import numpy as np

# Introduction

My objective is to find the following Hamiltonian ground state. 

$$
\mathbf{H}
=
\begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 0 & -1  & 0 \\ 0 & -1 & 0  & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}$$

In other words, we want to figure out the eigenvector with the minimum eigenvalue. In order to solve this problem we'll use a **variational quantum eigensolver** (VQE).  

*With a simple 4x4 matrix it is easier to find the ground state with any numerical python library (ex: numpy). However, I will use this simple matrix to learn how to implement VQE and I will use numpy in order to verify, if VQE gets to the ground state.*

In [4]:
matrix = np.array([[1,0,0,0],
                   [0,0,-1,0],
                   [0,-1,0,0],
                   [0,0,0,1]])

## 1st step Descomposition:


In [3]:
np.linalg.eig(matrix)

(array([ 1., -1.,  1.,  1.]),
 array([[ 0.        ,  0.        ,  1.        ,  0.        ],
        [-0.70710678,  0.70710678,  0.        ,  0.        ],
        [ 0.70710678,  0.70710678,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]))

In [7]:
def HS(M1, M2):
    """Hilbert-Schmidt-Product of two matrices M1, M2"""
    return (np.dot(M1.conjugate().transpose(), M2)).trace()

def c2s(c):
    """Return a string representation of a complex number c"""
    if c == 0.0:
        return "0"
    if c.imag == 0:
        return "%g" % c.real
    elif c.real == 0:
        return "%gj" % c.imag
    else:
        return "%g+%gj" % (c.real, c.imag)

def decompose(H):
    """Decompose Hermitian 4x4 matrix H into Pauli matrices"""
    from numpy import kron
    sx = np.array([[0, 1],  [ 1, 0]], dtype=np.complex128)
    sy = np.array([[0, -1j],[1j, 0]], dtype=np.complex128)
    sz = np.array([[1, 0],  [0, -1]], dtype=np.complex128)
    id = np.array([[1, 0],  [ 0, 1]], dtype=np.complex128)
    S = [id, sx, sy, sz]
    labels = ['I', 'sigma_x', 'sigma_y', 'sigma_z']
    for i in range(4):
        for j in range(4):
            label = labels[i] + ' \otimes ' + labels[j]
            a_ij = 0.25 * HS(kron(S[i], S[j]), H)
            if a_ij != 0.0:
                print ("%s\t*\t( %s )" % (c2s(a_ij), label))

In [8]:
np.diag([0,0,0,1])

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 1]])

In [9]:
decompose(matrix)

0.5	*	( I \otimes I )
-0.5	*	( sigma_x \otimes sigma_x )
-0.5	*	( sigma_y \otimes sigma_y )
0.5	*	( sigma_z \otimes sigma_z )


In [10]:
from qiskit.chemistry import FermionicOperator

In [11]:
m = FermionicOperator(h1=matrix)

In [12]:
m.h1

array([[ 1,  0,  0,  0],
       [ 0,  0, -1,  0],
       [ 0, -1,  0,  0],
       [ 0,  0,  0,  1]])

In [21]:
a=m.mapping('jordan_wigner')
print(a.print_details())

IIII	(1+0j)
IIIZ	(-0.5+0j)
IYYI	(-0.5+0j)
IXXI	(-0.5+0j)
ZIII	(-0.5+0j)



In [22]:
a=m.mapping('parity')
print(a.print_details())

IIII	(1+0j)
IIIZ	(-0.5+0j)
IZXZ	(0.5+0j)
IIXI	(-0.5+0j)
ZZII	(-0.5+0j)



In [26]:
a=m.mapping('bravyi_kitaev')
print(a.print_details())

IIII	(1+0j)
IIIZ	(-0.5+0j)
IYYZ	(-0.5+0j)
IXXI	(-0.5+0j)
ZZZI	(-0.5+0j)



In [25]:
a=m.mapping('bksf')
print(a.print_details())

Operator is empty.
